# Preliminary

The purpose of preliminaries is to use historical data to detect a profitable pattern

In [ ]:
import pandas as pd
from tqdm import trange
from utils.read2df import read2df, unify_dfs

symbols = ['BTCUSDT', 'ETHUSDT']
freqs = {'3m': 3}
period = 1000

date_format = '%Y-%m-%d'
start_date = '2023-01-01'
end_date = '2023-12-31'

In [ ]:
%%capture
if symbols is None:
    !python binance-public-data/python/download-kline.py \
        -i {" ".join(list(freqs.keys()))} -startDate {start_date} -endDate {end_date} -t spot -skip-daily 1
else:
    !python binance-public-data/python/download-kline.py \
        -s {" ".join(symbols)} -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1

In [ ]:
dfs = read2df(symbols=symbols, freqs=freqs, marketType='spot')
tics, df = unify_dfs(dfs, symbols=symbols, period=period)

In [ ]:
df.iloc[2000:3000:100]

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

ax1.plot(df['datetime'], df['spread'], label='spread')
ax2.plot(df['datetime'], df['zscore'], label='zscore')

ax1.legend()
ax2.legend()
plt.suptitle(f'The Spread between {symbols[0]} and {symbols[1]}')

plt.xticks(rotation=45)
plt.show()

In [ ]:
cash = 10000
position, holdings = 0, [0, 0]
open_thre, clos_thre = 2, 0.1
tc = 0.0005
next_pos = 0

for i in trange(len(df)):
    price0 = df.iloc[i]['close0']
    price1 = df.iloc[i]['close1']
    max_unit0 = cash/price0
    max_unit1 = cash/price1

    if position==0 and df.iloc[i]['zscore']>=open_thre:
        holdings = [max_unit0*(1-tc), -max_unit1*(1-tc)]
        next_pos = -1
    elif position==0 and df.iloc[i]['zscore']<=-open_thre:
        holdings = [-max_unit0*(1-tc), max_unit1*(1-tc)]
        next_pos = 1
    elif position==-1 and df.iloc[i]['zscore']<=clos_thre:
        cash += (holdings[0] * price0 + holdings[1] * price1) * (1-tc)
        holdings = [0, 0]
        next_pos = 0
    elif position==1 and df.iloc[i]['zscore']>=-clos_thre:
        cash += (holdings[0] * price0 + holdings[1] * price1) * (1-tc)
        holdings = [0, 0]
        next_pos = 0

    position = next_pos

networth = cash + holdings[0] * price0 + holdings[1] * price1
print(f"networth: {networth}")